<a href="https://colab.research.google.com/github/stefan-lazarevski/vnp-24-25/blob/master/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [5]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: /content/ElectricCarData.csv
100% 8.20k/8.20k [00:00<00:00, 5.30MB/s]


### Import the required libraries

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [7]:
data = pd.read_csv('ElectricCarData.csv')

In [8]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            103 non-null    object 
 1   Model            103 non-null    object 
 2   AccelSec         103 non-null    float64
 3   TopSpeed_KmH     103 non-null    int64  
 4   Range_Km         103 non-null    int64  
 5   Efficiency_WhKm  103 non-null    int64  
 6   FastCharge_KmH   103 non-null    object 
 7   RapidCharge      103 non-null    object 
 8   PowerTrain       103 non-null    object 
 9   PlugType         103 non-null    object 
 10  BodyStyle        103 non-null    object 
 11  Segment          103 non-null    object 
 12  Seats            103 non-null    int64  
 13  PriceEuro        103 non-null    int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 11.4+ KB


In [11]:
data.isnull().sum() / len(data) * 100

,0
Brand,0.0
Model,0.0
AccelSec,0.0
TopSpeed_KmH,0.0
Range_Km,0.0
Efficiency_WhKm,0.0
FastCharge_KmH,0.0
RapidCharge,0.0
PowerTrain,0.0
PlugType,0.0


### Encode string variables

In [12]:
data_copy = data.copy()

In [ ]:
cat = ['Brand','Model', 'RapidCharge', 'PowerTrain', 'PlugType', 'BodyStyle', 'Segment', 'FastCharge_KmH']

In [25]:
encoder = OrdinalEncoder()
data_copy[['Brand','Model','RapidCharge','PowerTrain','PlugType','BodyStyle','Segment', 'FastCharge_KmH']] = encoder.fit_transform(data_copy[['Brand','Model','RapidCharge','PowerTrain','PlugType','BodyStyle','Segment', 'FastCharge_KmH']])

In [26]:
data_copy

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,30.0,46.0,4.6,233,450,161,50.0,1.0,0.0,2.0,7.0,3.0,5,55480
1,31.0,33.0,10.0,160,270,167,7.0,1.0,2.0,2.0,1.0,2.0,5,30000
2,23.0,0.0,4.7,210,400,181,36.0,1.0,0.0,2.0,2.0,3.0,5,56440
3,2.0,101.0,6.8,180,360,206,32.0,1.0,2.0,2.0,6.0,3.0,5,68040
4,9.0,78.0,9.5,145,170,168,2.0,1.0,2.0,2.0,1.0,1.0,4,32997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,20.0,6.0,7.5,160,330,191,21.0,1.0,1.0,2.0,1.0,2.0,5,45000
99,1.0,94.0,4.5,210,335,258,30.0,1.0,0.0,2.0,6.0,4.0,5,96050
100,20.0,8.0,5.9,200,325,194,21.0,1.0,0.0,2.0,1.0,2.0,5,50000
101,20.0,10.0,5.1,200,375,232,22.0,1.0,0.0,2.0,1.0,2.0,5,65000


## Split the dataset for training and testing in ratio 80:20

In [27]:
X = data_copy.drop('PriceEuro',axis=1)
Y = data_copy['PriceEuro']

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=True, test_size=0.2, random_state=0)

## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [29]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            103 non-null    float64
 1   Model            103 non-null    float64
 2   AccelSec         103 non-null    float64
 3   TopSpeed_KmH     103 non-null    int64  
 4   Range_Km         103 non-null    int64  
 5   Efficiency_WhKm  103 non-null    int64  
 6   FastCharge_KmH   103 non-null    float64
 7   RapidCharge      103 non-null    float64
 8   PowerTrain       103 non-null    float64
 9   PlugType         103 non-null    float64
 10  BodyStyle        103 non-null    float64
 11  Segment          103 non-null    float64
 12  Seats            103 non-null    int64  
 13  PriceEuro        103 non-null    int64  
dtypes: float64(9), int64(5)
memory usage: 11.4 KB


In [30]:
dtr_model = DecisionTreeRegressor(max_depth=5, min_samples_split=10, max_features=6, random_state=6)

In [31]:
dtr_model.fit(X_train, Y_train)

DecisionTreeRegressor(max_depth=5, max_features=6, min_samples_split=10,
                      random_state=6)

## Predict the outcomes for X test

In [32]:
dtr_pred= dtr_model.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [33]:
dt_mse = mean_squared_error(Y_test, dtr_pred)
dt_r2 = r2_score(Y_test, dtr_pred)

print("DecisionTreeRegressor Performance:")
print(f"Mean Squared Error: {dt_mse}")
print(f"R-squared: {dt_r2}")

DecisionTreeRegressor Performance:
Mean Squared Error: 1207338649.4066916
R-squared: 0.5254904777944929


## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [34]:
xgb_model = XGBRegressor(max_depth=10, n_estimators=50, learning_rate=0.1, objective='reg:squarederror')

In [35]:
xgb_model.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Predict the outcomes for X test

In [36]:
xgb_pred = xgb_model.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [37]:
xgb_mse = mean_squared_error(Y_test, xgb_pred)
xgb_r2 = r2_score(Y_test, xgb_pred)

print("\nXGBRegressor Performance:")
print(f"Mean Squared Error: {xgb_mse}")
print(f"R-squared: {xgb_r2}")


XGBRegressor Performance:
Mean Squared Error: 1219719137.4020991
R-squared: 0.5206246972084045


## Compare the performances of both model for at least three regression metircs

In [38]:
print("\nComparison of Model Performances:")
print(f"DecisionTreeRegressor - MSE: {dt_mse}, R2: {dt_r2}")
print(f"XGBRegressor - MSE: {xgb_mse}, R2: {xgb_r2}")


Comparison of Model Performances:
DecisionTreeRegressor - MSE: 1207338649.4066916, R2: 0.5254904777944929
XGBRegressor - MSE: 1219719137.4020991, R2: 0.5206246972084045


In [39]:
if dt_r2 >= xgb_r2:
    print("Decision Tree better")
else:
    print("XGBoost better")


Decision Tree better
